In [ ]:
import numpy as np
import os
import sys
import cv2
import matplotlib.pyplot as plt
import open3d as o3d
%matplotlib inline 

src_path = os.path.abspath("../..")
if src_path not in sys.path:
    sys.path.append(src_path)
%load_ext autoreload
from dataset.kitti_odometry_dataset import KittiOdometryDataset, KittiOdometryDatasetConfig
from dataset.filters.filter_list import FilterList
from dataset.filters.kitti_gt_mo_filter import KittiGTMovingObjectFilter
from dataset.filters.range_filter import RangeFilter
from dataset.filters.apply_pose import ApplyPose

import scipy
from scipy.spatial.distance import cdist
from normalized_cut import normalized_cut
from ncuts_utils import ncuts_chunk,kDTree_1NN_feature_reprojection_colors, get_merge_pcds
from dataset_utils import * 
from point_cloud_utils import get_pcd, transform_pcd, kDTree_1NN_feature_reprojection, remove_isolated_points, get_subpcd, get_statistical_inlier_indices, merge_chunks_unite_instances
from aggregate_pointcloud import aggregate_pointcloud
from visualization_utils import generate_random_colors, color_pcd_by_labels
from sam_label_distace import sam_label_distance
from chunk_generation import subsample_positions, chunks_from_pointcloud, indices_per_patch, tarl_features_per_patch, image_based_features_per_patch, dinov2_mean, get_indices_feature_reprojection
from metrics_class import Metrics

Here we define the dataset depending on kitti sequence!

In [ ]:
DATASET_PATH = os.path.join('/Users/cedric/Datasets/semantic_kitti/')
SEQUENCE_NUM = 7

ind_start = 0
ind_end = 1100
minor_voxel_size = 0.05
major_voxel_size = 0.35
chunk_size = np.array([25, 25, 25]) #meters
overlap = 3 #meters
ground_segmentation_method = 'patchwork' 
NCUT_ground = False 
out_folder_ncuts = 'test_data/'

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=True)

out_folder = 'pcd_preprocessed/'
if os.path.exists(out_folder) == False : 
        os.makedirs(out_folder)


Now we aggregate a large point cloud based on (ind_start, ind_end)

In [ ]:
process_and_save_point_clouds(dataset,ind_start,ind_end,minor_voxel_size=minor_voxel_size,
                              major_voxel_size=major_voxel_size,icp=False,
                              out_folder=out_folder,sequence_num=SEQUENCE_NUM,
                              ground_segmentation_method=ground_segmentation_method)


In [ ]:
##load data if already stored 
pcd_ground_minor, pcd_nonground_minor,\
	all_poses, T_pcd, first_position,kitti_labels = load_and_downsample_point_clouds(out_folder,SEQUENCE_NUM,minor_voxel_size,\
                                                                     ground_mode=ground_segmentation_method)

In [ ]:
'''
pcd_new = o3d.geometry.PointCloud()
pts_num = 1000000
pcd_new.points = o3d.utility.Vector3dVector(np.asarray(pcd_nonground_minor.points)[:pts_num])

map_labelled = color_pcd_by_labels(pcd_new,\
                kitti_labels['panoptic_nonground'][:pts_num].reshape(-1,1))

o3d.visualization.draw_geometries([map_labelled])
#o3d.io.write_point_cloud('labelled_map07.pcd',map_labelled)
'''

Now we subsample the poses based on a voxel_size

In [ ]:
poses, positions, \
sampled_indices_local, sampled_indices_global = subsample_and_extract_positions(all_poses,ind_start=ind_start)

Now we can split the point cloud into chunks based on a tbd chunk_size

In [ ]:
pcd_nonground_chunks, pcd_ground_chunks,\
pcd_nonground_chunks_major_downsampling, pcd_ground_chunks_major_downsampling, \
indices,indices_ground, center_positions, \
center_ids, chunk_bounds, kitti_labels = chunk_and_downsample_point_clouds(pcd_nonground_minor, pcd_ground_minor, T_pcd, positions, 
                                                            first_position, sampled_indices_global, chunk_size=chunk_size, 
                                                            overlap=overlap, major_voxel_size=major_voxel_size,kitti_labels=kitti_labels)

In [ ]:
import sklearn
from sklearn.cluster import DBSCAN

def DBSCAN_clustering_logic(cur_pcd, pcd_all, eps=1.0, min_samples=100):
    """
    Perform DBSCAN clustering on the point cloud data.

    :param cur_pcd: Current point cloud for clustering.
    :param pcd_all: All point cloud data.
    :param eps: The maximum distance between two samples for one to be considered as in the neighborhood of the other.
    :param min_samples: The number of samples in a neighborhood for a point to be considered as a core point.
    :return: Cluster labels for each point in the point cloud.
    """
    not_road_points = np.asarray(cur_pcd.points)
    clustering = DBSCAN(eps=eps, min_samples=min_samples).fit(not_road_points)
    labels_not_road = clustering.labels_
    colors_gen = generate_random_colors(500)
    
    # Reproject cluster labels to the original point cloud size
    cluster_labels = np.ones((len(pcd_all.points), 1)) * -1
    labels_non_ground = kDTree_1NN_feature_reprojection(cluster_labels, pcd_all, labels_not_road.reshape(-1,1), cur_pcd)
    colors = np.zeros((labels_non_ground.shape[0],3))
    unique_labels = list(np.unique(labels_non_ground))
    for j in unique_labels:
            cur_idcs = np.where(labels_non_ground == j)[0]
            
            colors[cur_idcs] = np.array(colors_gen[unique_labels.index(j)])
    pcd_all.colors = o3d.utility.Vector3dVector(colors / 255.)
    return pcd_all

def dbscan_clustering(pcd_chunks_major_downsampled, pcds, center_ids,ground_clouds):
    labels_clustering = []
    for i in range(len(pcd_chunks_major_downsampled)):
        cur_pcd = pcd_chunks_major_downsampled[i]
        pcd_all = pcds[i]
        ground_cloud = ground_clouds[i]
        cluster_labels = DBSCAN_clustering_logic(cur_pcd, pcd_all)  # Implement your DBSCAN logic here
        ground_labels = np.ones((np.asarray(ground_cloud.points).shape[0],1)) * -1
        cluster_labels = np.concatenate((cluster_labels,ground_labels),0)
        labels_clustering.append(cluster_labels)
        
    return labels_clustering


In [ ]:
alpha = 0.0
theta = 0.0
beta = 1.0
gamma = 0.0
proximity_threshold = 1.0
out_dbscan = 'out_dbscan/'
if os.path.exists(out_dbscan) == False : 
        os.makedirs(out_dbscan)

out_kitti = 'out_kitti/'
if os.path.exists(out_kitti) == False : 
        os.makedirs(out_kitti)

patchwise_indices = indices_per_patch(T_pcd, center_positions, positions, first_position, sampled_indices_global, chunk_size)
out_data = []
for sequence in range(len(center_ids))[:2]:
        if NCUT_ground == False : 
                
                merged_chunk,file_name, pcd_chunk, pcd_chunk_ground, inliers_ground = ncuts_chunk(dataset,indices,pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=False,sequence=sequence,
                        patchwise_indices=patchwise_indices)
                
                kitti_labels['ground']['panoptic'][sequence] = kitti_labels['ground']['panoptic'][sequence][inliers_ground]
                
                name = file_name.split('/')[-1]
                o3d.io.write_point_cloud(file_name, pcd_chunk + pcd_chunk_ground , write_ascii=False, compressed=False, print_progress=False)
                
                pcd_dbscan = DBSCAN_clustering_logic(pcd_nonground_chunks_major_downsampling[sequence],
                                                pcd_nonground_chunks[sequence],
                                                eps=0.6, min_samples=10)
                
                kitti_chunk = color_pcd_by_labels(pcd_chunk,kitti_labels['nonground']['panoptic'][sequence].reshape(-1,))
                o3d.io.write_point_cloud(out_dbscan + name, pcd_dbscan + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                o3d.io.write_point_cloud(out_kitti + name, kitti_chunk + pcd_chunk_ground, write_ascii=False, compressed=False, print_progress=False)
                
        else : 
                obstacle_out,file_name = ncuts_chunk(dataset,indices,pcd_nonground_chunks,pcd_ground_chunks,
                        pcd_nonground_chunks_major_downsampling,
                        pcd_nonground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=True,sequence=sequence,
                        patchwise_indices=patchwise_indices)
                
                
                ground_out,file_name = ncuts_chunk(dataset,indices_ground,pcd_ground_chunks,None,
                        pcd_ground_chunks_major_downsampling,
                        pcd_ground_minor,T_pcd,center_positions,center_ids,
                        positions,first_position,sampled_indices_global,
                        chunk_size=chunk_size,major_voxel_size=major_voxel_size,
                        alpha=alpha,beta=beta,gamma=gamma,theta=theta,
                        proximity_threshold=proximity_threshold,
                        out_folder=out_folder_ncuts,ground_mode=True,sequence=sequence,
                        patchwise_indices=patchwise_indices)

                o3d.io.write_point_cloud(file_name, obstacle_out + ground_out, write_ascii=False, compressed=False, print_progress=False)

                print("Pointcloud written to file")

        
        
        

In [ ]:
def get_merge_pcds(out_folder_ncuts):
        point_clouds = []

        # List all files in the folder
        files = os.listdir(out_folder_ncuts)
        files.sort()

        # Filter files with a .pcd extension
        pcd_files = [file for file in files if file.endswith(".pcd")]
        print(pcd_files)
        # Load each point cloud and append to the list
        for pcd_file in pcd_files:
                file_path = os.path.join(out_folder_ncuts, pcd_file)
                point_cloud = o3d.io.read_point_cloud(file_path)
                point_clouds.append(point_cloud)
        return point_clouds

In [ ]:
point_clouds = get_merge_pcds(out_folder_ncuts)
point_clouds_dbscan = get_merge_pcds(out_dbscan)
point_clouds_kitti = get_merge_pcds(out_kitti)
merge = merge_chunks_unite_instances(point_clouds)
merge_dbscan = merge_chunks_unite_instances(point_clouds_dbscan)
merge_kitti = merge_chunks_unite_instances(point_clouds_kitti)

o3d.io.write_point_cloud(out_folder + "merge_part.pcd", merge, write_ascii=False, compressed=False, print_progress=False)
o3d.io.write_point_cloud(out_folder + "merge_part_dbscan.pcd", merge_dbscan, write_ascii=False, compressed=False, print_progress=False)
o3d.io.write_point_cloud(out_folder + "merge_part_kitti.pcd", merge_kitti, write_ascii=False, compressed=False, print_progress=False)

In [ ]:
merge = o3d.io.read_point_cloud(out_folder + 'merge_part.pcd')
merge_dbscan = o3d.io.read_point_cloud(out_folder + 'merge_part_dbscan.pcd')
merge_kitti = o3d.io.read_point_cloud(out_folder + 'merge_part_kitti.pcd')

In [ ]:

o3d.visualization.draw_geometries([merge_kitti])




unique_colors, labels_ncuts = np.unique(np.asarray(merge.colors), axis=0, return_inverse=True)
unique_colors, labels_dbscan = np.unique(np.asarray(merge_dbscan.colors), axis=0, return_inverse=True)
unique_colors, labels_kitti = np.unique(np.asarray(merge_kitti.colors),axis=0, return_inverse=True)

metrics_ncuts = Metrics(name='ncuts')
metrics_dbscan = Metrics(name='dbscan')
metrics_test = Metrics(name='test')
metrics_ncuts.update_stats(labels_ncuts,labels_kitti)
metrics_dbscan.update_stats(labels_dbscan,labels_kitti)

metrics_dbscan.compute_all_aps()
metrics_ncuts.compute_all_aps()

#merge_vis = color_pcd_by_labels(merge,labels)
#o3d.visualization.draw_geometries([merge_vis])


In [ ]:
np.concatenate(kitti_labels['nonground']['panoptic'][:2]).shape

In [ ]:
kitti_labels['nonground']['panoptic'][:len(point_clouds)] + [-2] * (num_points - nonground_points)

In [ ]:
DATASET_PATH = os.path.join('/Users/cedric/Datasets/semantic_kitti/')
SEQUENCE_NUM = 7

dataset = create_kitti_odometry_dataset(DATASET_PATH,SEQUENCE_NUM,ncuts_mode=False)


In [ ]:
from open3d.pipelines import registration
import numpy as np 
merge.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,max_nn=200))

for i in range(70,72):
	local_pcd = o3d.geometry.PointCloud()
	local_pcd.points = o3d.utility.Vector3dVector(dataset[i].point_cloud[:, :3])
	
	
	## label visualization 
	labeled_pcd = o3d.geometry.PointCloud()
	labeled_pcd.points = o3d.utility.Vector3dVector(dataset[i].point_cloud[:, :3])
	labeled_pcd.colors = o3d.utility.Vector3dVector(np.vstack([0,0,0] for i in range(np.asarray(labeled_pcd.points).shape[0])))
	panoptic_labels = dataset[i].panoptic_labels # semantics + panoptics combined 
	semantic_labels = dataset[i].semantic_labels
	instance_labels = dataset[i].instance_labels
	intensity = dataset[i].intensity
	
	transform = dataset.get_pose(i)
	local_pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.5,max_nn=200))
	reg_p2l = registration.registration_icp(local_pcd, merge, 0.9, transform, registration.TransformationEstimationPointToPlane(), registration.ICPConvergenceCriteria(max_iteration=1000))
	transform = reg_p2l.transformation
	local_pcd.transform(transform)
	
	local_pcd.normals = o3d.utility.Vector3dVector([])
	
	local_pcd.paint_uniform_color([0, 0, 0])
	
	
	
	
	colors, labels,local_labels = kDTree_1NN_feature_reprojection_colors(np.asarray(local_pcd.colors), local_pcd, np.asarray(merge.colors), merge,panoptic_labels, max_radius=0.2)
	labeled_pcd = color_pcd_by_labels(labeled_pcd,labels.reshape(-1,))
	local_pcd.colors = o3d.utility.Vector3dVector(colors)
	#o3d.io.write_point_cloud("test.pcd", local_pcd, write_ascii=False, compressed=False, print_progress=False)
	
	
	
	labeled_pcd.translate([0,120,0])
	o3d.visualization.draw_geometries([local_pcd,labeled_pcd])
	unique_colors, labels = np.unique(colors, axis=0, return_inverse=True)
	unseen_mask = np.all(colors == [1,0,0], axis=1)
	labels[unseen_mask] = -1
	street_mask = np.all(colors == [0,0,0], axis=1)
	labels[street_mask] = 1
	
	#np.savez('output_files/7_tarl/ ' + str(i) + '.npz',labels=labels)

	

	#np.savez('output_files/7_original/' + str(i) + '.npz',ncut_labels=local_labels,kitti_labels=labels,pts=np.asarray(local_pcd.points),           
	#intensities=intensity,kitti_labels_semantic=semantic_labels,kitti_labels_instance=instance_labels)


In [ ]:
import numpy as np 
with np.load('output_files/7_original/0.npz') as data:
            xyz = data['pts'].astype(np.float)
            labels = data['ncut_labels'].astype(np.int32)  
            kitti_labels = data['kitti_labels']
            intensity = data['intensities']
            
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd = color_pcd_by_labels(pcd,labels)
o3d.visualization.draw_geometries([pcd])
        